# Step 1 - Introduction and Data Splitting
## Goals in this Notebook
To introduce the reader with the project and create a path for the project to be executed. Then, split the data at hand into train and test.
## Introduction
TODO
## Project Plan
This plan will be iteratively modified to reflect new findings in the dataset and potential improvements in the proof-of-concept.
### Project Structure
**Step 1. Test / Train data split**
- The first step is to split the data to have sufficient data for testing the models which are the ultimate goal of this project.

**Step 2. EDA**
- Get acquainted with the data at hand. Not all of this process will be reflected in the notebooks due to the vastness of features, but a part of it will. I will start with univariate analysis, checking what data and at what proportions is present in the various tables.

**Step 3. Identify a potential business problem**
- During EDA, a potential problem should emerge.

**Step 4. Prepare data for the problem**
- Find out what data could be used to train a model to solve the identified problem.

**Step 5. Train and select the best model to solve the problem**

**(Go to Step 3 until several potential problems are solved)**

**Step 6. Test and deploy the selected models**
### Assumptions
- We are most interested about the customer credit history which happened during the last year. That being said, we will still use data from the whole history at hand.
- TARGET feature is defined as "client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample". In this project I will not try to guess the X and Y but rather try to create features that could define a client with potential payment difficulties.
### Potential Business Problems
- Predict the "TARGET" feature in the application_train table. This feature indicates whether a client has payment problems.
- Predict the "NAME_CONTRACT_STATUS" in the previous_application table. This feature indicates whether the application was approved, cancelled, refused.
- Predict the "CODE_REJECT_REASON" in the previous_application table. This feature indicates why the application was rejected.

## Data Splitting

### Imports

In [1]:
import functions
from sklearn.model_selection import train_test_split

### Read-in Data, Convert to Parquet

In [2]:
path_app = r"data\application_train.csv"
path_bur = r"data\bureau.csv"
path_bur_bal = r"data\bureau_balance.csv"
path_prev_app = r"data\previous_application.csv"
path_cash = r"data\POS_CASH_balance.csv"
path_inst = r"data\installments_payments.csv"
path_cred = r"data\credit_card_balance.csv"

df_app = functions.csv_to_parquet(path_app)
df_bur = functions.csv_to_parquet(path_bur)
df_bur_bal = functions.csv_to_parquet(path_bur_bal)
df_prev_app = functions.csv_to_parquet(path_prev_app)
df_cash = functions.csv_to_parquet(path_cash)
df_inst = functions.csv_to_parquet(path_inst)
df_cred = functions.csv_to_parquet(path_cred)

### Check for Duplicates

In [4]:
df_app.duplicated().value_counts()

False    307511
Name: count, dtype: int64

In [5]:
df_bur.duplicated().value_counts()

False    1716428
Name: count, dtype: int64

In [6]:
df_bur_bal.duplicated().value_counts()

False    27299925
Name: count, dtype: int64

In [7]:
df_prev_app.duplicated().value_counts()

False    1670214
Name: count, dtype: int64

In [8]:
df_cash.duplicated().value_counts()

False    10001358
Name: count, dtype: int64

In [9]:
df_cred.duplicated().value_counts()

False    3840312
Name: count, dtype: int64

In [10]:
df_inst.duplicated().value_counts()

False    13605401
Name: count, dtype: int64

There are no duplicates.

### Split df_app

Since df_app sits at the top of the hierarchy of all the tables, I would like to split it according to this table.

One thing that could be improved here: the application_test.csv has no "Target" column, but the data could still potentially be merged in and used for some other purposes.

In [3]:
df_app_train, df_app_test = train_test_split(
    df_app, test_size=0.05, shuffle=True, random_state=42
)

### Handle Train Split on Other Tables

All other tables will be split based on df_app ids. Bureau is easy to split. Cash, inst and cred dataframes are harder, since they can be handled directly by SK_ID_CURR or through df_app.

In [5]:
curr_ids = df_app_train["SK_ID_CURR"]

# Handle Bureau
df_bur_train = df_bur[df_bur["SK_ID_CURR"].isin(curr_ids)]
bureau_ids = df_bur_train["SK_ID_BUREAU"]
df_bur_bal_train = df_bur_bal[df_bur_bal["SK_ID_BUREAU"].isin(bureau_ids)]

# Handle cash, inst, cred tables directly through SK_ID_CURR
df_cash_curr = df_cash[df_cash["SK_ID_CURR"].isin(curr_ids)]
df_inst_curr = df_inst[df_inst["SK_ID_CURR"].isin(curr_ids)]
df_cred_curr = df_cred[df_cred["SK_ID_CURR"].isin(curr_ids)]

# Handle cash, inst, cred tables through prev_app table
df_prev_app_train = df_prev_app[df_prev_app["SK_ID_CURR"].isin(curr_ids)]
prev_ids = df_prev_app_train["SK_ID_PREV"]
df_cash_prev = df_cash[df_cash["SK_ID_PREV"].isin(prev_ids)]
df_inst_prev = df_inst[df_inst["SK_ID_PREV"].isin(prev_ids)]
df_cred_prev = df_cred[df_cred["SK_ID_PREV"].isin(prev_ids)]

In [6]:
print("Shape of cash table when handled directly:", df_cash_curr.shape)
print("Shape of cash table when handled indirectly:", df_cash_prev.shape)

Shape of cash table when handled directly: (8117603, 8)
Shape of cash table when handled indirectly: (7841360, 8)


When handled indirectly, we get a smaller dataset, which would imply that we don't have all the previous applications in the prev_app table.

We can also check if the "_curr" tables are supersets of the "_prev" tables.

In [7]:
cols = ["SK_ID_CURR", "SK_ID_PREV"]
functions.check_if_superset(df_cash_curr, df_cash_prev, cols, "cash_curr", "cash_prev")
functions.check_if_superset(df_inst_curr, df_inst_prev, cols, "inst_curr", "inst_prev")
functions.check_if_superset(df_cred_curr, df_cred_prev, cols, "cred_curr", "cred_prev")

cash_curr table is a superset of cash_prev
inst_curr table is a superset of inst_prev
cred_curr table is a superset of cred_prev


Since all the "curr" tables are supersets, we'll only save them and discard "prev" tables.

In [8]:
df_cash_train = df_cash_curr.copy()
df_inst_train = df_inst_curr.copy()
df_cred_train = df_cred_curr.copy()

### Handle Test Split on Other Tables

Now we can also define the test set.

In [9]:
curr_ids = df_app_test["SK_ID_CURR"]

# Handle Bureau
df_bur_test = df_bur[df_bur["SK_ID_CURR"].isin(curr_ids)]
bureau_ids = df_bur_test["SK_ID_BUREAU"]
df_bur_bal_test = df_bur_bal[df_bur_bal["SK_ID_BUREAU"].isin(bureau_ids)]

# Handle cash, inst, cred tables directly through SK_ID_CURR
df_cash_test = df_cash[df_cash["SK_ID_CURR"].isin(curr_ids)]
df_inst_test = df_inst[df_inst["SK_ID_CURR"].isin(curr_ids)]
df_cred_test = df_cred[df_cred["SK_ID_CURR"].isin(curr_ids)]

# Handle prev_app
df_prev_app_test = df_prev_app[df_prev_app["SK_ID_CURR"].isin(curr_ids)]

### Save as Parquet

In [11]:
df_app_train.to_parquet(r"data\train\df_app.parquet", engine="pyarrow")
df_app_test.to_parquet(r"data\test\df_app.parquet", engine="pyarrow")
df_bur_train.to_parquet(r"data\train\df_bur.parquet", engine="pyarrow")
df_bur_test.to_parquet(r"data\test\df_bur.parquet", engine="pyarrow")
df_bur_bal_train.to_parquet(r"data\train\df_bur_bal.parquet", engine="pyarrow")
df_bur_bal_test.to_parquet(r"data\test\df_bur_bal.parquet", engine="pyarrow")
df_prev_app_train.to_parquet(r"data\train\df_prev_app.parquet", engine="pyarrow")
df_prev_app_test.to_parquet(r"data\test\df_prev_app.parquet", engine="pyarrow")
df_cash_train.to_parquet(r"data\train\df_cash.parquet", engine="pyarrow")
df_cash_test.to_parquet(r"data\test\df_cash.parquet", engine="pyarrow")
df_inst_train.to_parquet(r"data\train\df_inst.parquet", engine="pyarrow")
df_inst_test.to_parquet(r"data\test\df_inst.parquet", engine="pyarrow")
df_cred_train.to_parquet(r"data\train\df_cred.parquet", engine="pyarrow")
df_cred_test.to_parquet(r"data\test\df_cred.parquet", engine="pyarrow")